# Divergence RSI - Higher High but Lower RSI or Lower Low but Higher RSI - Analytics

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir) + '/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765
...,...,...,...,...,...
2025-02-14 14:15:00,1343.0,1343.0,1340.3,1341.3,7141
2025-02-14 14:20:00,1340.9,1341.9,1340.5,1341.4,4593
2025-02-14 14:25:00,1341.1,1342.5,1340.7,1342.5,4207


In [5]:
data['RSI'] = ta.rsi(data["Close"], length=14)

In [6]:
# Identify Higher High (HH) and Lower RSI
data['HighestHigh'] = data['High'].shift(1).rolling(20).max()
data['HH'] = data['High'] > data['HighestHigh'] + 0.1
data['Lower_RSI'] = data['RSI'] < data['RSI'].shift(1).rolling(20).max()

# Find divergence signals
data['Bearish_Divergence'] = data['HH'] & data['Lower_RSI']

In [7]:
data[data.Bearish_Divergence == True]

,Open,High,Low,Close,Volume,RSI,HighestHigh,HH,Lower_RSI,Bearish_Divergence
Date,,,,,,,,,,
2018-08-13 13:55:00,947.5,950.2,947.5,949.9,2232,65.900156,949.6,True,True,True
2018-08-14 10:15:00,956.6,957.3,956.3,957.1,1335,71.564741,957.0,True,True,True
2018-08-14 13:30:00,956.6,957.4,956.6,957.3,2459,63.522868,956.9,True,True,True
2018-08-15 10:30:00,961.0,962.3,961.0,962.1,2060,63.015550,961.5,True,True,True
2018-08-15 13:55:00,959.4,959.9,959.0,959.2,2356,54.207342,959.6,True,True,True
...,...,...,...,...,...,...,...,...,...,...
2025-02-12 13:30:00,1339.9,1340.7,1339.8,1339.8,3092,58.758702,1340.2,True,True,True
2025-02-13 11:15:00,1333.4,1333.9,1332.5,1332.9,4527,53.900114,1333.6,True,True,True
2025-02-13 13:20:00,1335.9,1336.6,1335.1,1335.6,6156,61.803374,1335.9,True,True,True


In [8]:
# Identify Lower Low (LL) and Higher RSI
data['LowestLow'] = data['Low'].shift(1).rolling(20).min()
data['LL'] = data['Low'] < data['LowestLow'] - 0.1
data['Higher_RSI'] = data['RSI'] > data['RSI'].shift(1).rolling(20).min()

# Find divergence signals
data['Bullish_Divergence'] = data['LL'] & data['Higher_RSI']

In [9]:
data[data.Bullish_Divergence == True]

,Open,High,Low,Close,Volume,RSI,HighestHigh,HH,Lower_RSI,Bearish_Divergence,LowestLow,LL,Higher_RSI,Bullish_Divergence
Date,,,,,,,,,,,,,,
2018-08-14 10:50:00,952.8,954.4,952.2,953.7,2135,52.803395,957.3,False,True,False,952.6,True,True,True
2018-08-15 13:10:00,955.4,955.9,954.7,955.9,2478,41.461730,962.3,False,True,False,955.2,True,True,True
2018-08-15 14:10:00,956.1,956.2,953.3,954.1,4459,40.047054,959.9,False,True,False,954.7,True,True,True
2018-08-16 09:25:00,942.2,942.4,940.8,940.8,1870,24.656511,959.9,False,True,False,941.0,True,True,True
2018-08-16 09:30:00,940.6,941.8,940.6,941.8,1634,27.864289,959.9,False,True,False,940.8,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-13 09:20:00,1329.7,1330.3,1328.5,1330.2,3555,30.755026,1340.7,False,True,False,1329.3,True,True,True
2025-02-13 10:10:00,1329.4,1329.8,1327.4,1327.9,5342,29.619595,1337.6,False,True,False,1328.5,True,True,True
2025-02-13 10:15:00,1327.9,1328.1,1326.5,1327.0,5126,27.725339,1337.6,False,True,False,1327.4,True,True,True


In [10]:
data[data.index >= '2025-02-14 14:15:00']

,Open,High,Low,Close,Volume,RSI,HighestHigh,HH,Lower_RSI,Bearish_Divergence,LowestLow,LL,Higher_RSI,Bullish_Divergence
Date,,,,,,,,,,,,,,
2025-02-14 14:15:00,1343.0,1343.0,1340.3,1341.3,7141,41.346671,1347.9,False,True,False,1341.1,True,True,True
2025-02-14 14:20:00,1340.9,1341.9,1340.5,1341.4,4593,41.729821,1347.5,False,True,False,1340.3,False,True,False
2025-02-14 14:25:00,1341.1,1342.5,1340.7,1342.5,4207,45.827369,1347.2,False,True,False,1340.3,False,True,False
2025-02-14 14:30:00,1342.5,1342.5,1342.5,1342.5,150,45.827369,1346.8,False,True,False,1340.3,False,True,False
2025-02-14 14:45:00,1341.8,1341.8,1341.8,1341.8,5684,43.662439,1346.7,False,True,False,1340.3,False,True,False
